<a href="https://colab.research.google.com/github/Shirleyhywl/hello-world/blob/master/Copy_of_YueHu_W207_ALSY_FinalProject_Baseline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data and Plan

## Andrew Webb, Laura Chutny, Suzy Choi, Yue Hu
### 1. From Kaggle
Each predicted keypoint is specified by an (x,y) real-valued pair in the space of pixel indices. There are 15 keypoints, which represent the following elements of the face:

left_eye_center  
right_eye_center  
left_eye_inner_corner  
left_eye_outer_corner  
right_eye_inner_corner  
right_eye_outer_corner  
left_eyebrow_inner_end  
left_eyebrow_outer_end  
right_eyebrow_inner_end  
right_eyebrow_outer_end  
nose_tip  
mouth_left_corner  
mouth_right_corner  
mouth_center_top_lip  
mouth_center_bottom_lip  

Left and right here refers to the point of view of the subject.  

In some examples, some of the target keypoint positions are missing (encoded as missing entries in the csv, i.e., with nothing between two commas).

The input image is given in the last field of the data files, and consists of a list of pixels (ordered by row), as integers in (0,255). The images are 96x96 pixels.

#### Data files - have been uploaded to GitHub Repo
- training.csv: list of training 7049 images. Each row contains the (x,y) coordinates for 15 keypoints, and image data as row-ordered list of pixels.  
- test.csv: list of 1783 test images. Each row contains ImageId and image data as row-ordered list of pixels
- submissionFileFormat.csv: list of 27124 keypoints to predict. Each row contains a RowId, ImageId, FeatureName, Location. FeatureName are "left_eye_center_x," "right_eyebrow_outer_end_y," etc. Location is what you need to predict.


### 2. Plan
1. Data Cleaning
- TBD based on the tutorial
2. Split data - use training set and split into train, dev and test.

3. kNN; MN Naive Bayes; Logistic Regression --> run and compare as base

4. Maybe run

5. Then learn and run Neural Net to classify

### 3. Baseline
Due Week 10: (July 8)
- Plan for project
- Do Tutorial
- EDA / Cleaning
- More detailed plan around project (maybe a pseudocode type plan)
- check out Google colab - using this
- Theano - install - no need when using colab

Week 11/12 (July 15 and 22)

Week 12 - Checkin with Instructor  (July 22)

Week 14 - Project due (August 5)



### 4. Setup

In [0]:
# ADD OTHER IMPORTS HERE AS NECESSARY
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import sys
import csv
import os

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_openml

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# Theano imports
import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs
np.random.seed(0) # Setting random seed


cpu
float64


In [0]:
print("python version =",sys.version)
print("Pandas Version = ",pd.__version__)
print("Numpy Version = ",np.__version__)

python version = 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
Pandas Version =  0.24.2
Numpy Version =  1.16.4


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.getcwd()

'/content'

### 5. Data Import and Cleaning
 - There is a Train and Test sample in the data from Kaggle
 - We should split the Train data into Train and Dev - and optimize on Dev; Save Test for the very last step in each method.
 - Select an appropriate scoring method (per the Kaggle contest and maybe one other as well)

###Train Data (training.csv)

*  There is total 31 columns (features) and 7049 rows (images)
  
    *  30 columns for 15 predicted keypoint pair specified by a (x,y) real-value pair in the space of pixel indices
    * 1 column for 96*96 pixel input image consists a list of pixels (ordered by row), as integers in (0,255) in string type



In [0]:
def Data_PreProcess(train = True):
  """ Loads data based on the purpose of training or testing (train=True, test=FALSE).
  Tranform last column of imagine information from string to numpy array.
  Finally, normalized keypoint pairs based on each keypoint average."""
  
  # load data 
  if train: 
    data = pd.read_csv('drive/My Drive/FinalProject_w207/training.csv')
    df = pd.DataFrame(data)
  else:
    data = pd.read_csv('drive/My Drive/FinalProject_w207/test.csv')
    df = pd.DataFrame(data)
  print('There is %i columns (features)' % df.shape[0])
  print('There is %i rows (images)' % df.shape[1])
  print(df.count())
  
  # remove the rows which contain NA value
  df = df.dropna()
  print('There is %i rows (images) after remove NA values' % df.shape[0])
  
  # transform image data and load as X value
  df['Image']=df['Image'].apply(lambda X: np.fromstring(X, sep=' ')) # change string in 'image' column to nparray
  image= np.vstack(df['Image'].to_numpy())   # Convert 'image'column to nparray
  X = image/255
  
  
  # Normalize keypoint pairs and load as Y value
  keypoints = df[list(df.columns)[:-1]].to_numpy()
  mean = df[list(df.columns)[:-1]].mean(axis=1).to_numpy()
  mean_array = np.repeat(mean.reshape(df.shape[0],1),30,axis =1)
  Y= (keypoints-mean_array)/mean_array
  
  return X,Y

X,Y = Data_PreProcess()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
    X.shape, X.min(), X.max()))
print("Y.shape == {}; Y.min == {:.3f}; Y.max == {:.3f}".format(
    Y.shape, Y.min(), Y.max()))

    

There is 7049 columns (features)
There is 31 rows (images)
left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y 

In [0]:
# Create Dev Data out of the Train data set.

### Single Hidden Layer
Create a neural net with a single hidden layer

*   **Activation Function** ReLU (Rectified Linear Unit) y= max(0,X)
https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7
*  **Cost Function** Mean Squared Error (MSE)
*  **Optimization Method** Nesterov_momentum gradient descent batch optimization with learning rate of 0.01 and momentum parameters of 0.9. The maximum iteration is set to 400.







In [0]:
pip install git+https://github.com/Lasagne/Lasagne

  Cloning https://github.com/Lasagne/Lasagne to /tmp/pip-req-build-lbbyt_8p
  Running command git clone -q https://github.com/Lasagne/Lasagne /tmp/pip-req-build-lbbyt_8p
  Stored in directory: /tmp/pip-ephem-wheel-cache-59gv8zml/wheels/c4/20/90/9f7242c381402829c5918261e3eb51a87bc1d8521456749b57
Successfully built Lasagne


In [0]:
pip install nolearn

     |████████████████████████████████| 327kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/9c/b3/e8/a863ca29687669deabe68012f333d5c5bd6c9b5a3bea4f2538
  Stored in directory: /root/.cache/pip/wheels/f1/aa/d7/7968a6696737f947ab854dc3394186a8fcfbd8aedcc8c66d07
  Stored in directory: /root/.cache/pip/wheels/61/c5/d7/74a2e7afea611ed0469c95bfb9d21f89dd73f1f4f9ecd7a784
Successfully built nolearn gdbn gnumpy


In [0]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch, None translates to a variable batch size
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # activation function is Rectifier
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1, # print out information during training
    )

X, Y = Data_PreProcess()
net1.fit(X, Y)

SkipTest: ignored

### 6. kNN Classification & Scoring

Make sure to add a commentary at the end with a summary of this section including what was done and what the result was (on Dev and Test)

### 7. Multinomial Naive Bayes Classification & Scoring
Make sure to add a commentary at the end with a summary of this section including what was done and what the result was (on Dev and Test)

### 8. Logistic Regression Classification & Scoring
Make sure to add a commentary at the end with a summary of this section including what was done and what the result was (on Dev and Test)

### 9. Neural Net Classification & Scoring  

Note - must binarize the labels before processing.
Make sure to add a commentary at the end with a summary of this section including what was done and what the result was (on Dev and Test)

In [0]:
# Sample Code from Week 7 Tutorial - DON'T RUN AS IS, DATA SETS WON'T WORK
# Single layer NN

# Sample Binarization code:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data
train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size
print('Classes = %d' %(numClasses))

NameError: ignored

In [0]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, value
z = (np.random.randn(*(numFeatures, numClasses))*0.01)
print(z.shape, type(z))
z2 = np.asarray(z)
print(z2.shape, type(z2))
w = theano.shared((np.random.randn(*(numFeatures, numClasses))*.01))
print(w.ndim,type(w),type(w.get_value()))
print(w.dtype)
print(w.nonzero(), w.shape)

In [0]:
## (2) Model
# Theano objects accessed with standard Python variables
# using Softmax - i.e. basically just a multi-class version of sigmoid activation
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)
print(y_hat,y_hat.dtype, y_hat.ndim, type(y_hat))

In [0]:
## (3) Cost function
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))
print(cost, cost.dtype, cost.ndim, type(cost))

In [0]:
## (4) Objective (and solver)

alpha = 0.01
gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) # computes cost, then runs update
y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        cost = train(train_data[0:len(train_data)], train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescent(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

Next Steps - do some/all/none of the following:
- Try Stochastic and Mini-Batch 
- Try 2 and 3 layer models
- Try using a different activation - like rectifier and/or max pooling
- Try adding noise (like Dropouts) - can increase generalization and training speeds

**FINALLY**: *Convolutional NN*

- This is likely the prize - developed for image processing
- Need to reprocess data - uses 2D images (not 1D)
- Go to last section of Week7 Tutorial for code on Convolutional NNs
- Possibly do a generative model to create images and further train the network

### 10. Conclusions  

Make sure to write up summary of different methods, results based on Dev and then on Test (maybe present in table). 